https://www.riksdagen.se/sv/dokument-lagar/dokument/motion/nu-ar-det-dags-for-sverige--sverigedemokraternas_H9023938

In [174]:
import re
from io import StringIO
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [175]:
multiplier = 1_000

In [192]:
search_url = 'https://data.riksdagen.se/dokumentlista/'
params = {
    'sok': 'utgiftsområde',
    'doktyp': 'mot',
    'rm': '2021/22',
    'parti': 'SD',
    'sort': 'rel',
    'sortorder': 'desc',
    'utformat': 'json',
    'a': 's'}

resp = requests.get(search_url, params=params)
data = resp.json()
docs = pd.DataFrame(data['dokumentlista']['dokument'])
docs = docs[['titel', 'undertitel', 'rm', 'dokument_url_html']]

In [193]:
docs['Utgiftsområde'] = docs.titel.str.extract('Utgiftsområde (\d+)')
docs['Utgiftsområde'] = docs['Utgiftsområde'].astype(int)
docs = docs.sort_values('Utgiftsområde').reset_index(drop=True)

In [194]:
def collect_tables(url):
    resp = requests.get(url)
    text = resp.text
    soup = BeautifulSoup(text)
    h3s = soup.select('table~h3')
    tables = soup.select('h3+table')
    frames = []

    for h3, table in zip(h3s, tables):
        headline = h3.text.strip()
        if not re.match('\d+:\d+', headline):
            continue
        frame = pd.read_html(StringIO(str(table)), header=0, thousands=' ')[0]
        frame['Anslag'] = headline
        # Misspelled header
        frame = frame.rename(columns={'Beräknat 2023.1': 'Beräknat 2024'})
        # Other misspelled header
        if 'Förslag 2021' in frame:
            frame = frame.rename(columns={'Beräknat 2023': 'Beräknat 2024',
                                          'Beräknat 2022': 'Beräknat 2023',
                                          'Förslag 2021': 'Förslag 2022'})
        frames.append(frame)
    
    try:
        df = pd.concat(frames)
    except ValueError:
        df = None
    return df

In [195]:
frames = []

for _, doc in docs.iterrows():
    frame = collect_tables(f'http:{doc.dokument_url_html}')
    if frame is None:
        print(doc.Utgiftsområde)
        continue
    frame['Utgiftsområde'] = doc.Utgiftsområde
    frames.append(frame)

2
5


In [196]:
df = pd.concat(frames)

In [199]:
# Fix error
df['Beräknat 2024'] = df['Beräknat 2024'].fillna(df['Beräknat 2024 4'])
df = df.drop('Beräknat 2024 4', axis=1)

In [200]:
# Special case for utgiftsområde 2
# https://riksdagen.se/sv/dokument-lagar/dokument/motion/utgiftsomrade-2-samhallsekonomi-och_H9023906

df = df.append([{
    'Anslag': '1:11 Finansinspektionen',
    'Förslag 2022': '5000',
    'Beräknat 2023': '5000',
    'Beräknat 2024': '5000',
    'Utgiftsområde': 2,
}])

In [201]:
# Special case for utgiftsområde 5
# https://riksdagen.se/sv/dokument-lagar/dokument/motion/utgiftsomrade-2-samhallsekonomi-och_H9023906

df = df.append([
    {
        'Anslag': '1:3 Nordiskt samarbete',
        'Förslag 2022': '10000',
        'Beräknat 2023': '10000',
        'Beräknat 2024': '10000',
        'Utgiftsområde': 5
    },
    {
        'Anslag': '1:5 Inspektionen för strategiska produkter',
        'Förslag 2022': '1000',
        'Beräknat 2023': '1000',
        'Beräknat 2024': '1000',
        'Utgiftsområde': 5
    },
    {
        'Anslag': '1:9 Svenska institutet',
        'Förslag 2022': '10000',
        'Beräknat 2023': '10000',
        'Beräknat 2024': '10000',
        'Utgiftsområde': 5
    },])

In [202]:
# Hack to sum up too detailed rows
df = df.drop(['Avvikelse från regeringen', 'Unnamed: 0'], axis=1)
df.columns = [x.replace('Förslag ', '').replace('Beräknat ', '') for x in df.columns]
for col in ['2022', '2023', '2024']:
    df[col] = df[col].fillna(0).astype(str)
    df[col] = df[col].str.split('.', expand=True)[0]
    df[col] = df[col].str.replace('±0', '0', regex=False)
    df[col] = df[col].str.replace('\s+', '', regex=True)
    df[col] = df[col].str.replace('−', '-')
    df[col] = df[col].str.replace('–', '-')
    df[col] = df[col].astype(int) * multiplier

df = df.reset_index(drop=True)
_map = df[['Anslag', 'Utgiftsområde']].drop_duplicates().set_index('Anslag')['Utgiftsområde'].to_dict()
df = df.groupby('Anslag', as_index=False).sum().drop('Utgiftsområde', axis=1)
df['Utgiftsområde'] = df.Anslag.map(_map)
df = df.sort_values('Utgiftsområde')

In [203]:
df['Namn'] = df.Anslag.str.split('\d+:\d+', expand=True)[1].str.strip()
df['Anslag'] = df.Anslag.str.extract('^(\d+:\d+) ')

In [208]:
df.to_csv('../data/budgetmotion-2022-sd.csv', index=False)